This notebook generates a one-day dataframe of interpolated pv + nwp data.

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

First, lets load in the NWP and PV one-day datas.

In [2]:
path_to_nwp = './nwpdata/20220101.zarr'
path_to_pv = './pvdata/pv/italy/PV_PVOutput.org_Italy_PVOutput_Italy_systems_metadata.csv'

In [3]:
nwp_data = xr.open_dataset(path_to_nwp, engine='zarr', chunks='auto')
pv_data = pd.read_csv(path_to_pv)

In [4]:
nwp_data

<xarray.Dataset>
Dimensions:    (time: 18, latitude: 121, longitude: 113)
Coordinates:
  * latitude   (latitude) float32 58.0 57.75 57.5 57.25 ... 28.5 28.25 28.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 27.25 27.5 27.75 28.0
  * time       (time) datetime64[ns] 2022-01-01T05:00:00 ... 2022-01-01T22:00:00
Data variables: (12/60)
    alnid      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    alnip      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvd      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvp      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    bld        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    ...         ...
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    u10n       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    uvb        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    wind       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-11-26 09:57:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [5]:
pv_data

,Unnamed: 0,name,system_DC_capacity_W,address,num_panels,panel_capacity_W_each,panel_brand,num_inverters,inverter_capacity_W,inverter_brand,...,shade,install_date,latitude,longitude,status_interval_minutes,secondary_num_panels,secondary_panel_capacity_W_each,secondary_orientation,secondary_array_tilt_degrees,system_id
0,0,Pergola Fotovoltaica,9165,NaN,39,235,LG,1,10000,Power One PVI 10.0 OUTD-S,...,No,2011-02-28,36.715286,15.091528,5,0,0,NaN,NaN,24000
1,1,Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,No,NaN,36.718600,15.090700,5,0,0,NaN,NaN,48316
2,2,Tetto Fotovoltaico Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,No,NaN,36.718600,15.090700,5,0,0,NaN,NaN,41556
3,3,Scalecci,13230,96017.0,63,210,Centrosolar,1,12500,Power One PVI 12.5 OUTD FS,...,No,2013-02-25,36.734759,15.098036,5,0,0,NaN,NaN,24001
4,4,Marzamemi-FV,2990,96017.0,13,230,LG 230W,1,3000,Power One Aurora PVI 3.0,...,Medium,2011-04-28,36.747195,15.108724,5,0,0,NaN,NaN,24760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1320,Test 11,50,39031.0,1,50,NaN,1,50,Energy Monitor,...,No,NaN,46.795800,11.936400,5,0,0,NaN,NaN,60986
1321,1321,OptisolarSos,1000,33020.0,4,250,NaN,1,4000,OptiSolar,...,No,2014-11-04,46.518650,12.835684,5,0,0,NaN,NaN,33148
1322,1322,sonneundsee,5280,NaN,22,240,Schueco,1,5500,Kostal,...,No,2012-09-01,46.687427,13.975522,10,0,0,NaN,NaN,20925
1323,1323,DV,382200,39049.0,1274,300,Sunrise,26,5500,Solutronic,...,No,2010-12-31,46.896300,11.447600,10,0,0,NaN,NaN,79743


Next, we check the min and max of the latitude and longitude just to make sure they fall within our cropped NWP data lat/lon range. Looks like they do.

In [6]:
pv_data[['latitude', 'longitude']].min()

latitude     36.715286
longitude     6.738385
dtype: float64

In [7]:
pv_data[['latitude', 'longitude']].max()

latitude     46.896300
longitude    18.422746
dtype: float64

In [8]:
nwp_data

<xarray.Dataset>
Dimensions:    (time: 18, latitude: 121, longitude: 113)
Coordinates:
  * latitude   (latitude) float32 58.0 57.75 57.5 57.25 ... 28.5 28.25 28.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 27.25 27.5 27.75 28.0
  * time       (time) datetime64[ns] 2022-01-01T05:00:00 ... 2022-01-01T22:00:00
Data variables: (12/60)
    alnid      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    alnip      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvd      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    aluvp      (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    bld        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    ...         ...
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    u10n       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    uvb        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
    wind       (time, latitude, longitude) float32 dask.array<chunksize=(18, 121, 113), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-11-26 09:57:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

Next, lets create our mapping -> By rounding the lat/lon to the nearest .25 of a lat/lon

In [9]:
pv_data['nearest_latitude'] = round(pv_data['latitude'] * 4) /4
pv_data['nearest_longitude'] = round(pv_data['longitude'] * 4) /4

In [10]:
def map_val(value: float) -> float:
    """
    Maps a lat/lon value to its interpolated NWP point.
    """
    return round(value * 4) / 4

In [11]:
pv_data

,Unnamed: 0,name,system_DC_capacity_W,address,num_panels,panel_capacity_W_each,panel_brand,num_inverters,inverter_capacity_W,inverter_brand,...,latitude,longitude,status_interval_minutes,secondary_num_panels,secondary_panel_capacity_W_each,secondary_orientation,secondary_array_tilt_degrees,system_id,nearest_latitude,nearest_longitude
0,0,Pergola Fotovoltaica,9165,NaN,39,235,LG,1,10000,Power One PVI 10.0 OUTD-S,...,36.715286,15.091528,5,0,0,NaN,NaN,24000,36.75,15.00
1,1,Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,36.718600,15.090700,5,0,0,NaN,NaN,48316,36.75,15.00
2,2,Tetto Fotovoltaico Saiazza,14000,96018.0,70,200,Deluxe,1,12500,Aurora,...,36.718600,15.090700,5,0,0,NaN,NaN,41556,36.75,15.00
3,3,Scalecci,13230,96017.0,63,210,Centrosolar,1,12500,Power One PVI 12.5 OUTD FS,...,36.734759,15.098036,5,0,0,NaN,NaN,24001,36.75,15.00
4,4,Marzamemi-FV,2990,96017.0,13,230,LG 230W,1,3000,Power One Aurora PVI 3.0,...,36.747195,15.108724,5,0,0,NaN,NaN,24760,36.75,15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1320,Test 11,50,39031.0,1,50,NaN,1,50,Energy Monitor,...,46.795800,11.936400,5,0,0,NaN,NaN,60986,46.75,12.00
1321,1321,OptisolarSos,1000,33020.0,4,250,NaN,1,4000,OptiSolar,...,46.518650,12.835684,5,0,0,NaN,NaN,33148,46.50,12.75
1322,1322,sonneundsee,5280,NaN,22,240,Schueco,1,5500,Kostal,...,46.687427,13.975522,10,0,0,NaN,NaN,20925,46.75,14.00
1323,1323,DV,382200,39049.0,1274,300,Sunrise,26,5500,Solutronic,...,46.896300,11.447600,10,0,0,NaN,NaN,79743,47.00,11.50


Next, lets load in our PV time series data (which has data on voltage, temperature, and power) and cropping out a single day of the data that lines up with our NWP data. This is the data we'll be combining with our NWP data. Note that the previous PV dataframe we loaded was our PV metadata which has info where the different sites are located.

In [12]:
pv_timeseries = pd.read_csv('pvdata/pv/italy/pv_time_series.csv')
pv_timeseries

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage
0,10441,2015-01-01 08:00:00,108.000000,2.933333,231.000000
1,10441,2015-01-01 08:15:00,573.333333,3.233333,229.333333
2,10441,2015-01-01 08:30:00,780.000000,6.000000,230.333333
3,10441,2015-01-01 08:45:00,1002.666667,3.666667,232.000000
4,10441,2015-01-01 09:00:00,1264.333333,8.133333,231.666667
...,...,...,...,...,...
31547136,9664,2022-10-01 18:30:00,70.333333,30.633333,226.033333
31547137,9664,2022-10-01 18:45:00,20.333333,29.966667,226.833333
31547138,9664,2022-10-01 19:00:00,1.333333,29.200000,226.766667
31547139,9664,2022-10-01 19:15:00,0.000000,28.566667,227.000000


In [13]:
pv_timeseries['timestamp'] = pd.to_datetime(pv_timeseries['timestamp'])

In [14]:
start_time = nwp_data['time'][0].values
end_time = nwp_data['time'][-1].values

pv_day_data = pv_timeseries[(pv_timeseries['timestamp'] >= start_time) & (pv_timeseries['timestamp'] <= end_time)]

In [15]:
pv_day_data

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage
119555,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000
119556,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000
119557,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667
119558,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667
119559,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000
...,...,...,...,...,...
31532436,9664,2022-01-01 16:30:00,58.333333,20.000000,227.033333
31532437,9664,2022-01-01 16:45:00,10.000000,19.366667,226.633333
31532438,9664,2022-01-01 17:00:00,0.000000,18.633333,227.266667
31532439,9664,2022-01-01 17:15:00,0.000000,18.066667,226.800000


As some pre-checks, lets ensure that the # of system IDs in our time series data lines up with, or at least is less than the # of system IDs in the PV metadata. \\
Once again, note that
- pv_day_data is our time series data
- pv_data is the pv metadata

In [16]:
len(pv_day_data['system_id'].unique())

293

In [17]:
len(pv_data['system_id'].unique())

1325

In [18]:
from tqdm import tqdm

# These are the NWP variable names that we will be merging with our pv data
# We use this in the inner for loop where we are adding the NWP dataframe to the PV dataframe.
nwp_variable_names = list(nwp_data.variables)

pv_day_data['lat'] = None
pv_day_data['lon'] = None
pv_day_data['time'] = None
for i, row in tqdm(pv_day_data.iterrows()):

    # We only have hourly NWP data, so if the PV data is for something other than hourly,
    # then we want to ignore that row (it'll be NaN)
    timestamp = row['timestamp']
    if timestamp.minute != 0:
        continue

    # First, lets get the latitude and longitude of the specific row from its system_id
    system_id = row['system_id']
    lat = map_val(pv_data[pv_data['system_id'] == system_id]['latitude']).values[0]
    lon = map_val(pv_data[pv_data['system_id'] == system_id]['longitude']).values[0]

    # Next, we want to check if we've already inserted the same data into the dataframe.
    # This will reduce the time that this function runs for.
    cached_data = pv_day_data[(pv_day_data['lat'] == lat) & (pv_day_data['lon'] == lon) & (pv_day_data['timestamp'] == timestamp)]
    if len(cached_data) != 0:
        pv_day_data.loc[i, nwp_variable_names] = cached_data.iloc[0][nwp_variable_names]
        continue

    # Otherwise, we will perform a lookup in the NWP data, which is quite time consuming.
    data = nwp_data.sel(time=timestamp, latitude=lat, longitude=lon)
    for var in nwp_variable_names:
        val = data[var].values.item()
        pv_day_data.loc[i, var] = val

    # Cache the data
    pv_day_data.loc[i, 'lat'] = lat
    pv_day_data.loc[i, 'lon'] = lon
    pv_day_data.loc[i, 'timestamp'] = timestamp

C:\Users\Bill\AppData\Local\Temp\ipykernel_9948\233533475.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_day_data['lat'] = None
C:\Users\Bill\AppData\Local\Temp\ipykernel_9948\233533475.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_day_data['lon'] = None
C:\Users\Bill\AppData\Local\Temp\ipykernel_9948\233533475.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

Finally, lets do the interpolation and add our NWP data to our PV time series data.

In [19]:
pv_day_data

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage,lat,lon,time,alnid,alnip,...,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
119555,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119556,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119557,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667,43.75,11.25,1641027600000000000,0.185514,0.177517,...,860905.375,-950268.6875,-923276.6250,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
119558,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119559,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31532436,9664,2022-01-01 16:30:00,58.333333,20.000000,227.033333,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31532437,9664,2022-01-01 16:45:00,10.000000,19.366667,226.633333,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31532438,9664,2022-01-01 17:00:00,0.000000,18.633333,227.266667,None,None,1641056400000000000,0.178663,0.169128,...,0.000,-943413.5625,-936445.1875,-0.773845,-1.763546,-0.315648,0.000000,-0.718974,0.030087,NaN
31532439,9664,2022-01-01 17:15:00,0.000000,18.066667,226.800000,None,None,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
pv_day_data = pv_day_data.drop(['lat', 'lon', 'timestamp'], axis=1)

In [21]:
pv_day_data.to_csv('processed_pv_data_jan1_2023_new.csv', index=False)

Finally, lets perform a sanity check - lets make sure all of the rows in the optimized interpolation algorithm are the same as the ones in the unoptimized algorithm. Note that you don't have to include this when you add it to the data pipeline!

In [22]:
pv_day_data_old = pd.read_csv('processed_pv_data_jan1_2023.csv')

In [29]:
pv_day_data_old

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage,alnid,alnip,aluvd,aluvp,bld,...,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
0,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667,0.185514,0.177517,0.042215,0.041346,605.50,...,860905.375,-950268.6875,-923276.6250,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
3,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11795,9664,2022-01-01 16:30:00,58.333333,20.000000,227.033333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11796,9664,2022-01-01 16:45:00,10.000000,19.366667,226.633333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11797,9664,2022-01-01 17:00:00,0.000000,18.633333,227.266667,0.178663,0.169128,0.055082,0.053102,522.75,...,0.000,-943413.5625,-936445.1875,-0.773845,-1.763546,-0.315648,0.000000,-0.718974,0.030087,NaN
11798,9664,2022-01-01 17:15:00,0.000000,18.066667,226.800000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
pv_day_data[:50]

,system_id,instantaneous_power_W,temperature_C,voltage,time,alnid,alnip,aluvd,aluvp,bld,...,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
119555,10441,2.500000,NaN,223.500000,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119556,10441,13.333333,NaN,223.000000,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119557,10441,25.333333,NaN,220.666667,1641027600000000000,0.185514,0.177517,0.042215,0.041346,605.5000,...,860905.375,-950268.6875,-923276.6250,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
119558,10441,51.666667,NaN,217.666667,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119559,10441,70.333333,NaN,215.000000,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119560,10441,81.666667,NaN,215.666667,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119561,10441,123.666667,NaN,214.666667,1641031200000000000,0.185504,0.177508,0.042215,0.041346,584.8125,...,1288547.500,-955879.0000,-944538.3125,0.178086,0.248374,0.209821,79839.921875,0.633760,0.678995,NaN
119562,10441,144.666667,NaN,217.000000,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119563,10441,168.666667,NaN,220.000000,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119564,10441,204.666667,NaN,222.333333,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
pv_day_data_old[:50]

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage,alnid,alnip,aluvd,aluvp,bld,...,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
0,10441,2022-01-01 08:30:00,2.500000,NaN,223.500000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10441,2022-01-01 08:45:00,13.333333,NaN,223.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667,0.185514,0.177517,0.042215,0.041346,605.5000,...,860905.375,-950268.6875,-923276.6250,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
3,10441,2022-01-01 09:15:00,51.666667,NaN,217.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10441,2022-01-01 09:30:00,70.333333,NaN,215.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10441,2022-01-01 09:45:00,81.666667,NaN,215.666667,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10441,2022-01-01 10:00:00,123.666667,NaN,214.666667,0.185504,0.177508,0.042215,0.041346,584.8125,...,1288547.500,-955879.0000,-944538.3125,0.178086,0.248374,0.209821,79839.921875,0.633760,0.678995,NaN
7,10441,2022-01-01 10:15:00,144.666667,NaN,217.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10441,2022-01-01 10:30:00,168.666667,NaN,220.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10441,2022-01-01 10:45:00,204.666667,NaN,222.333333,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
print(pv_day_data.reset_index(drop=True).loc[2].values)

[10441 25.33333333333333 nan 220.66666666666663 1641027600000000000
 0.18551409244537354 0.17751729488372803 0.0422150194644928
 0.04134643077850342 605.5 82.217041015625 0.101806640625 34.107421875
 463087.0 0.0 2.3283064365386963e-10 0.0 278.6112365722656 3561.5439453125
 -0.7936562299728394 -0.7936402559280396 nan 230587.875 0.0
 -1.0065050446428359e-05 1.0 -24.094253540039062 -16.212221145629883 43.75
 0.9273648262023926 11.25 0.0 7.902272045612335e-06 6.011687219142914e-07
 0.0 -2.1827640011906624e-05 1.0000457763671875 281.8248291015625
 -57526.375 99387.140625 394589.375 574534.125 454513.625 663248.875
 -137295.46875 -305900.0625 1124332.75 913959.625 278.6341857910156
 0.9273648262023926 0.0 0.1739673614501953 8.377647399902344 1124237.375
 29.702913284301758 8.163973689079285e-06 694252.125 860905.375
 -950268.6875 -923276.625 -0.20413541793823242 0.685549259185791
 -0.23682773113250732 36905.359375 0.6941366195678711 1.3031364679336548
 nan]


In [44]:
pd.set_option('display.max_columns', None)


In [50]:
pd.DataFrame(pv_day_data.loc[119557]).T

,system_id,instantaneous_power_W,temperature_C,voltage,time,alnid,alnip,aluvd,aluvp,bld,blh,cape,cbh,cdir,cp,crr,csf,d2m,deg0l,dndza,dndzn,dwi,fdir,hcc,ie,ilspf,ishf,kx,latitude,lcc,longitude,lsf,lsp,lsrr,mcc,pev,ptype,skt,slhf,sp,ssr,ssrc,ssrd,ssrdc,str,strc,strd,strdc,t2m,tcc,tciw,tclw,tcwv,tisr,totalx,tp,tsr,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
119557,10441,25.333333,NaN,220.666667,1641027600000000000,0.185514,0.177517,0.042215,0.041346,605.5,82.217041,0.101807,34.107422,463087.0,0.0,0.0,0.0,278.611237,3561.543945,-0.793656,-0.79364,NaN,230587.875,0.0,-0.00001,1.0,-24.094254,-16.212221,43.75,0.927365,11.25,0.0,0.000008,0.000001,0.0,-0.000022,1.000046,281.824829,-57526.375,99387.140625,394589.375,574534.125,454513.625,663248.875,-137295.46875,-305900.0625,1124332.75,913959.625,278.634186,0.927365,0.0,0.173967,8.377647,1124237.375,29.702913,0.000008,694252.125,860905.375,-950268.6875,-923276.625,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN


In [51]:
pd.DataFrame(pv_day_data_old.loc[2]).T

,system_id,timestamp,instantaneous_power_W,temperature_C,voltage,alnid,alnip,aluvd,aluvp,bld,blh,cape,cbh,cdir,cp,crr,csf,d2m,deg0l,dndza,dndzn,dwi,fdir,hcc,ie,ilspf,ishf,kx,latitude,lcc,longitude,lsf,lsp,lsrr,mcc,pev,ptype,skt,slhf,sp,ssr,ssrc,ssrd,ssrdc,str,strc,strd,strdc,t2m,tcc,tciw,tclw,tcwv,tisr,totalx,tp,tsr,tsrc,ttr,ttrc,u10,u100,u10n,uvb,v10,v100,wind
2,10441,2022-01-01 09:00:00,25.333333,NaN,220.666667,0.185514,0.177517,0.042215,0.041346,605.5,82.217041,0.101807,34.107422,463087.0,0.0,0.0,0.0,278.611237,3561.543945,-0.793656,-0.79364,NaN,230587.875,0.0,-0.00001,1.0,-24.094254,-16.212221,43.75,0.927365,11.25,0.0,0.000008,0.000001,0.0,-0.000022,1.000046,281.824829,-57526.375,99387.140625,394589.375,574534.125,454513.625,663248.875,-137295.46875,-305900.0625,1124332.75,913959.625,278.634186,0.927365,0.0,0.173967,8.377647,1124237.375,29.702913,0.000008,694252.125,860905.375,-950268.6875,-923276.625,-0.204135,0.685549,-0.236828,36905.359375,0.694137,1.303136,NaN
